In [41]:
from final_project.database import db

collection = db.get_collection('reddit_word_frequency')
data = list(collection.find({}))

In [42]:
data = list(map(lambda x: {'word':x['word'], 'count':x['count']}, data))
print(data)

[{'word': 'idk', 'count': 5}, {'word': 'man', 'count': 6}, {'word': 'maybe', 'count': 15}, {'word': 'cracked', 'count': 2}, {'word': 'friends', 'count': 7}, {'word': 'personally', 'count': 9}, {'word': 'name', 'count': 19}, {'word': 'like', 'count': 107}, {'word': 'ten', 'count': 2}, {'word': 'dudes', 'count': 2}, {'word': 'know', 'count': 41}, {'word': 'schools', 'count': 12}, {'word': 'ucla', 'count': 2}, {'word': 'waterloo', 'count': 5}, {'word': 'columbia', 'count': 2}, {'word': 'michigan', 'count': 3}, {'word': 'canadian', 'count': 5}, {'word': 'probably', 'count': 25}, {'word': 'heard', 'count': 19}, {'word': 'interned', 'count': 4}, {'word': 'lol', 'count': 18}, {'word': 'particular', 'count': 3}, {'word': 'big', 'count': 26}, {'word': 'presence', 'count': 2}, {'word': 'intern', 'count': 14}, {'word': 'classes', 'count': 48}, {'word': 'hve', 'count': 2}, {'word': 'boughten', 'count': 2}, {'word': 'miself', 'count': 2}, {'word': 'primeium', 'count': 2}, {'word': 'noose', 'count':

In [43]:
import json
# Sort the list by word frequency
sorted_list = sorted(data, key=lambda x: x['count'], reverse=True)

# Print the sorted list
# print(json.dumps(sorted_list[:10], indent=4))
print(sorted_list[:10])

[{'word': "n't", 'count': 122}, {'word': 'like', 'count': 107}, {'word': 'get', 'count': 96}, {'word': 'work', 'count': 76}, {'word': 'time', 'count': 76}, {'word': 'internship', 'count': 72}, {'word': 'would', 'count': 64}, {'word': 'one', 'count': 61}, {'word': 'even', 'count': 55}, {'word': 'class', 'count': 52}]


In [30]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
# nltk.download('averaged_perceptron_tagger')

# words = ['apple', 'cat', 'jumped', 'over', 'the', 'lazy', 'dog']
# tokens = word_tokenize(' '.join(words))
tagged = pos_tag([e['word'] for e in sorted_list])
noun_set = [word for word, pos in tagged if pos.startswith('N')]

noun_freq = [e for e in sorted_list if e['word'] in noun_set]

print(tagged)

[('get', 'NN'), ('like', 'IN'), ('one', 'CD'), ("n't", 'RB'), ('internship', 'VB'), ('think', 'VB'), ('work', 'NN'), ('people', 'NNS'), ('got', 'VBD'), ('time', 'NN'), ('would', 'MD'), ('https', 'VB'), ('also', 'RB'), ('companies', 'NNS'), ('want', 'VBP'), ('company', 'NN'), ('teacher', 'RB'), ('well', 'RB'), ('offer', 'VB'), ('even', 'RB'), ('teachers', 'NNS'), ('know', 'VBP'), ('summer', 'NN'), ('much', 'JJ'), ('classes', 'NNS'), ('say', 'VBP'), ('interview', 'NN'), ('job', 'NN'), ('professor', 'IN'), ('year', 'NN'), ('need', 'VBP'), ('assignment', 'NN'), ('going', 'VBG'), ('na', 'JJ'), ('resume', 'JJ'), ('projects', 'NNS'), ('make', 'VBP'), ('saying', 'VBG'), ('extra', 'JJ'), ('lol', 'NN'), ('bad', 'JJ'), ('lot', 'NN'), ('something', 'NN'), ('good', 'JJ'), ('positions', 'NNS'), ('grade', 'VBP'), ('big', 'JJ'), ('getting', 'VBG'), ('gon', 'NN'), ('around', 'IN'), ('new', 'JJ'), ('first', 'JJ'), ('could', 'MD'), ('project', 'VB'), ('wrong', 'JJ'), ('still', 'RB'), ('experience', 'VB')

In [3]:
from final_project.clean_text.twitter_clean_text import preprocess_string

def get_reddit_comments():
    collection = db.get_collection('reddit_raw_data')
    data = list(collection.find({}))
    return data

def deduplicate_comments(comments):
    return list(set(comments))

In [54]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# model_name = 'mwkby/distilbert-base-uncased-sentiment-reddit-crypto'
model_name = 'Fan-s/reddit-tc-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

# Define the number of clusters
num_clusters = 20

# Define the words and their embeddings
# words = [e['word'] for e in sorted_list]
sentences = [preprocess_string(e['raw_text']) for e in get_reddit_comments()]
sentences = deduplicate_comments(sentences)
embeddings = []
# for word in words:
print("Processing sentences ...")
for sentence in sentences:
    input_ids = tokenizer.encode(sentence, return_tensors='pt', padding=True, truncation=True)  # Add special tokens
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs[2][1:]  # Get the embeddings from all hidden layers except the first
        embeddings.append(torch.cat(hidden_states, dim=-1).mean(dim=1).squeeze().numpy())

print("Clustering sentences ...")
# Perform k-means clustering on cosine similarity
similarity_matrix = cosine_similarity(embeddings)
kmeans = KMeans(n_clusters=num_clusters)
idx = kmeans.fit_predict(similarity_matrix)

# Print the words and their clusters
for i in range(num_clusters):
    cluster_words = []
    for j in range(len(sentences)):
        if idx[j] == i:
            cluster_words.append(sentences[j])
    print(f"Cluster {i+1}:")
    for t, sentence in enumerate(cluster_words):
        print(f"{t}. {sentence}")
        print()
    

Some weights of the model checkpoint at Fan-s/reddit-tc-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processing sentences ...
Clustering sentences ...
Cluster 1:
0. Yup, the other 15 credits is a 4 credit math course, 2 comp courses with lab (8 credits) and another comp/mathish course that’s 3 credits…. is that bad

1. AMZN. Applied September, interviewed October, got put on the waitlist, got an offer last week. You can DM if you want to talk more (so we don't spam a reddit thread)

2. It would be nice to have a part time job in school, you would make more year round this just one summer internship.

3. Just got APP offer today. Will be rejecting since I heard you cannot push for fall but I still am gonna give it one last email since I did the interview process with SDE and see if I can convert offer to Fall SDE.

4. Thanks! Got it right off of autotrader from a private seller who happened to be an old prof who babied the car for $30k. Definitely one of the best purchases I've made, good luck in your search!!

5. Do you think it's too late for me to even get in. I applied in January a

In [30]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch

def get_word_embedding(word, model):
    # Tokenize the word and convert to tensor
    tokens = tokenizer.encode(word, add_special_tokens=False)
    tokens_tensor = torch.tensor(tokens).unsqueeze(0)
#     print(tokens_tensor)
    # Get the output of the BERT model
    with torch.no_grad():
        outputs = model(tokens_tensor)

    # Extract the last hidden state of the BERT model output
    last_hidden_state = outputs.last_hidden_state

    # Get the word embedding for the first token (CLS token)
    word_embedding = last_hidden_state[:, 0, :]

    # Convert the word embedding to a numpy array
    word_embedding = word_embedding.numpy()
#     print(f"Word embedding for '{word}':")
#     print(word_embedding)
    
    return word_embedding

def cosine_similarity(u, v):
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    return dot_product / (norm_u * norm_v)

def get_embeddings_distance(base_emb, embeddings):
    dist = [cosine_similarity(base_emb[0], emb[0]) for emb in embeddings]
    return dist
    
def sort_embeddings_by_distance(base_emb, embeddings):
    get_embeddings_distance(base_emb, embeddings)
#     print(dist)
    return np.argsort(dist)

def extract_nouns(words):

    # Load the English language model
    nlp = spacy.load("en_core_web_sm")

    # Extract the nouns from the list of words
    nouns = []
    for word in words:
        doc = nlp(word)
        for token in doc:
            if token.pos_ == "NOUN":
                nouns.append(token.text)

    return nouns

def word_pair_distance(word1, word2, model):
    emb1 = get_word_embedding(word1, model)
    emb2 = get_word_embedding(word2, model)
    return cosine_similarity(emb1[0], emb2[0])

In [26]:
import re
# Load the pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
model_name = 'Fan-s/reddit-tc-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

pattern = re.compile("^[a-zA-Z]+$")

# Define the words to compare against
words = [e['word'] for e in sorted_list if pattern.match(e['word'])]
words = extract_nouns(words)
print("Convert words to embeddings ...")
print(words)
# Tokenize the words and encode them as input to the model
# tokens = tokenizer(words, return_tensors='pt', padding=True)
# print(tokens['input_ids'].shape)
# with torch.no_grad():
#     # Get the BERT embeddings for the input tokens
#     outputs = model(**tokens)
#     embeddings = outputs.last_hidden_state[:, 0, :].numpy()

# print("Calculate distance ...")
# base_emb = get_word_embedding('tech')

# # Find the word closest to "software engineer"
# target_word = 'software engineer'
# target_idx = words.index(target_word)
# similarity_scores = similarity_matrix[target_idx]
# most_similar_word_idx = np.argmax(similarity_scores)

# # Print the most similar word
# most_similar_word = words[most_similar_word_idx]
# print(f"The word most similar to '{target_word}' is '{most_similar_word}'.")


Some weights of the model checkpoint at Fan-s/reddit-tc-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'sorted_list' is not defined

In [15]:
# base_emb = get_word_embedding('technology', model)
# base_emb_2 = get_word_embedding('software engineer', model)
# base_emb_3 = get_word_embedding('job market', model)
# base_emb_4 = get_word_embedding('')
keywords = ['technology', 'software engineer', 'job market', 'interview']
# another_emb = get_word_embedding('')
# print(cosine_similarity(base_emb[0], another_emb[0]))

In [45]:
# Calculate the cosine similarity matrix for the word embeddings
embeddings = [get_word_embedding(word, model) for word in words]
# print("Finish convert word to embeddings")
def get_top_related_words(word, embeddings, words, model, top_k=300):
    base_emb = get_word_embedding(word, model)
    embedding_dist = np.array(get_embeddings_distance(base_emb, embeddings))

    # print(sorted(embedding_dist, reverse=False))
    # print(embedding_dist)
    sorted_embedding_index = np.argsort(-embedding_dist)

    sorted_words_by_distance = np.take(words, sorted_embedding_index)
    return sorted_words_by_distance[:top_k]

In [46]:
intersect = set.intersection(*[set(get_top_related_words(word, embeddings, words, model)) for word in keywords])
print(list(intersect))

['honors', 'opportunities', 'load', 'hackathons', 'postings', 'data', 'ban', 'guy', 'softie', 'questions', 'reason', 'press', 'reminders', 'interview', 'projectiles', 'repair', 'computer', 'macbook', 'plants', 'view', 'notagain', 'apology', 'comparison', 'staffs', 'evidence', 'sympathy', 'method', 'messages', 'format', 'lectures', 'keyboards', 'reality', 'teacher', 'degree', 'bit', 'anaology', 'qualifications', 'vision', 'frustration', 'software', 'environment', 'knowledge', 'rewards', 'lecturer', 'calc', 'process', 'loans', 'mind', 'internet', 'chips', 'overflow', 'kudos', 'search', 'lands', 'screen', 'undergrad', 'skill', 'engineering', 'eachother', 'wage', 'master', 'brutal', 'content', 'screens', 'accusation', 'office', 'development', 'stars', 'job', 'startups', 'dev', 'system', 'list', 'case', 'deal', 'experiencei', 'freelance', 'advice', 'pros', 'quality', 'undergrads', 'j', 'quizzes', 'company', 'wlbsource', 'cents', 'corporation', 'anecdotes', 'bed', 'effect', 'subject', 'frien

In [4]:
import pymongo

# Connect to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access a specific database
db = client["reddit_data"]

# Access a specific collection in the database
collection = db["reddit_post"]
data = list(collection.find({}))

In [3]:
texts = [post['selftext'] for post in data]

In [62]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Replace the following strings with the name or path of your desired model and tokenizer
model_name_or_path = "issifuamajeed/distilbert-base-uncased-finetuned-ner"
tokenizer_name_or_path = "issifuamajeed/distilbert-base-uncased-finetuned-ner"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
model = AutoModelForTokenClassification.from_pretrained(model_name_or_path)

# Create a NER pipeline using the loaded model and tokenizer
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer
)

# Define an example text to be processed
example_text = [text['selftext'] for text in data[-1000:]]

# Process the example text with the NER pipeline
ner_results = ner_pipeline(example_text)

print("Finish NER")
# Print the named entities identified by the pipeline
# print(ner_results)
for ner_result in ner_results:
    for entity in ner_result:
        print(f"{entity['word']} ({entity['entity']})")


Finish NER
p (B-PER)
. (B-PER)
s (B-PER)
google (B-ORG)
software (I-ORG)
engineering (I-ORG)
im (B-ORG)
##gur (B-ORG)
im (B-ORG)
##gur (B-ORG)
gi (B-ORG)
##th (B-ORG)
##ub (B-ORG)
cs (B-MISC)
cs (B-MISC)
cs (B-MISC)
q (B-MISC)
##a (B-ORG)
medium (B-ORG)
chen (B-ORG)
cs (B-MISC)
cs (B-MISC)
cs (B-MISC)
big (B-MISC)
4 (I-MISC)
community (B-ORG)
college (I-ORG)
un (B-ORG)
##i (B-ORG)
gi (B-MISC)
##th (B-ORG)
##ub (B-ORG)
gi (B-MISC)
##t (B-ORG)
cc (B-ORG)
gi (B-MISC)
##th (B-MISC)
##ub (B-MISC)
un (B-ORG)
##i (B-ORG)
c (B-LOC)
. (B-LOC)
s (B-LOC)
. (B-LOC)
gi (B-MISC)
##th (B-MISC)
##ub (B-MISC)
gi (B-MISC)
##th (B-ORG)
##ub (B-MISC)
sf (B-ORG)
##u (B-ORG)
canada (I-ORG)
cis (B-ORG)
fi (B-ORG)
##u (B-ORG)
ed (B-LOC)
##u (B-LOC)
cis (B-ORG)
fi (B-LOC)
##u (B-LOC)
ed (B-LOC)
cs (B-MISC)
cs (B-MISC)
##e (B-MISC)
goldman (B-ORG)
sachs (I-ORG)
angular (B-ORG)
cs (B-MISC)
fair (I-MISC)
fair (I-MISC)
c (B-LOC)
. (B-LOC)
s (B-LOC)
. (B-LOC)
un (B-ORG)
##i (B-ORG)
c (B-ORG)
. (B-ORG)
s (B-ORG)
. (

w (B-ORG)
##wu (B-ORG)
w (B-ORG)
##wu (B-ORG)
cs (B-MISC)
u (B-ORG)
##w (B-ORG)
seattle (B-LOC)
u (B-ORG)
##w (B-ORG)
cs (B-ORG)
school (I-ORG)
w (B-ORG)
##wu (B-ORG)
w (B-ORG)
##wu (B-ORG)
amazon (B-ORG)
microsoft (B-ORG)
google (B-ORG)
w (B-ORG)
##wu (B-ORG)
cs (B-ORG)
web (B-ORG)
##tech (B-ORG)
w (B-ORG)
##wu (B-ORG)
ed (B-ORG)
##u (B-ORG)
cs (B-MISC)
name (I-ORG)
cs (B-MISC)
ii (I-MISC)
j (B-MISC)
java (B-LOC)
cs (B-MISC)
##s (B-MISC)
j (B-MISC)
##s (B-MISC)
python (B-MISC)
java (I-MISC)
core (B-MISC)
java (I-MISC)
core (B-MISC)
java (I-MISC)
ii (I-MISC)
java (B-LOC)
us (B-LOC)
east (B-LOC)
coast (I-LOC)
cs (B-MISC)
cal (B-MISC)
##c (B-MISC)
2 (I-MISC)
cs (B-MISC)
cal (B-MISC)
##c (B-MISC)
2 (I-MISC)
cs (B-MISC)
cs (B-MISC)
##ic (B-MISC)
##ps (B-ORG)
##ci (B-ORG)
engineering (B-ORG)
##line (B-ORG)
sy (B-ORG)
##r (B-ORG)
ed (B-ORG)
##u (I-ORG)
cs (B-MISC)
baylor (B-ORG)
u (B-ORG)
##of (B-ORG)
##h (B-ORG)
california (B-ORG)
##e (I-ORG)
tech (B-ORG)
cs (B-MISC)
gi (B-ORG)
##th (B-ORG)

##wu (B-ORG)
u (B-ORG)
##w (B-ORG)
both (I-ORG)
##ell (I-ORG)
seattle (B-LOC)
gram (I-MISC)
##mer (I-MISC)
ios (B-MISC)
dev (I-MISC)
cs (B-MISC)
cs (B-MISC)
r (B-ORG)
cs (B-ORG)
red (B-ORG)
cs (B-MISC)
masters (I-MISC)
masters (B-MISC)
masters (B-MISC)
mit (B-ORG)
o (I-ORG)
##c (I-ORG)
##w (B-ORG)
cl (B-ORG)
##rs (B-ORG)
masters (B-MISC)
masters (B-MISC)
msc (B-MISC)
masters (B-MISC)
post (B-MISC)
- (B-MISC)
baccalaureate (B-MISC)
cal (B-ORG)
state (I-ORG)
school (I-ORG)
system (I-ORG)
open (B-MISC)
university (I-MISC)
- (B-MISC)
ba (B-MISC)
##cc (B-MISC)
tiny (B-ORG)
##ur (B-ORG)
##l (B-ORG)
tiny (B-ORG)
##ur (B-ORG)
##l (B-ORG)
french (B-MISC)
masters (B-MISC)
cs (B-MISC)
cs (B-MISC)
cs (B-MISC)
university (B-ORG)
of (I-ORG)
alabama (I-ORG)
at (I-ORG)
birmingham (I-ORG)
cu (B-LOC)
##da (B-LOC)
cm (B-ORG)
##u (B-ORG)
##er (B-MISC)
georgia (B-ORG)
tech (I-ORG)
cs (B-MISC)
biology (B-MISC)
english (B-MISC)
cs (B-LOC)
. (B-LOC)
mac (B-ORG)
##ales (B-ORG)
##ter (B-ORG)
ed (B-LOC)
##u (B-L

In [8]:
from tqdm import tqdm
results = []
for text in tqdm(orgs.keys(), desc="Analyzing texts"):
    result = ner_pipeline(text)
    results.append(result)

# Print the named entities for each text
for i, result in enumerate(results):
    print(f"Text {i+1}:")
    for entity in result:
        print(f" - Entity: {entity['entity']}, Text: {entity['word']}, Start: {entity['start']}, End: {entity['end']}")

Analyzing texts:   0%|                                 | 0/3719 [00:00<?, ?it/s]


NameError: name 'ner_pipeline' is not defined

In [57]:
filter_result = [lst for lst in results if lst]
filter_result[:100]
# len(results)

[[{'entity': 'I-ORG',
   'score': 0.9898545,
   'index': 1,
   'word': 'UC',
   'start': 0,
   'end': 2},
  {'entity': 'I-ORG',
   'score': 0.9879825,
   'index': 2,
   'word': '##SD',
   'start': 2,
   'end': 4}],
 [{'entity': 'I-ORG',
   'score': 0.99686074,
   'index': 1,
   'word': 'UC',
   'start': 0,
   'end': 2},
  {'entity': 'I-ORG',
   'score': 0.9911265,
   'index': 2,
   'word': 'Irvine',
   'start': 3,
   'end': 9}],
 [{'entity': 'I-ORG',
   'score': 0.48034742,
   'index': 4,
   'word': '&',
   'start': 17,
   'end': 18}],
 [{'entity': 'I-ORG',
   'score': 0.94191444,
   'index': 1,
   'word': 'CS',
   'start': 0,
   'end': 2}],
 [{'entity': 'I-ORG',
   'score': 0.8830567,
   'index': 1,
   'word': 'Q',
   'start': 0,
   'end': 1},
  {'entity': 'I-ORG',
   'score': 0.54712534,
   'index': 2,
   'word': '##ui',
   'start': 1,
   'end': 3},
  {'entity': 'I-ORG',
   'score': 0.800868,
   'index': 3,
   'word': '##x',
   'start': 3,
   'end': 4},
  {'entity': 'I-ORG',
   'scor

In [ ]:
"""
- Kafka process data every minute
- NER each text -> set of companies
- 
- Map each company with their texts -> Companies: [list of text]

"""

In [67]:
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

# texts = ["Apple is looking at buying U.K. startup for $1 billion", 
#          "Google announces new AI tool for healthcare industry",
#          "Amazon Web Services outage causes major disruptions for businesses"]
texts = [post['selftext'] for post in data]

orgs = {}
for text in tqdm(texts):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            orgs[ent.text] = 1 if ent.text not in orgs else orgs[ent.text] + 1
    

100%|█████████████████████████████████████| 85390/85390 [16:00<00:00, 88.88it/s]


In [81]:
sorted_org = dict(sorted(orgs.items(), key=lambda item: -item[1]))

(sorted_org)

{'CS': 12049,
 'Amazon': 3253,
 'Google': 1777,
 'GPA': 1665,
 'Microsoft': 1520,
 'AI': 788,
 'ML': 655,
 'Computer Science': 586,
 'SWE': 467,
 'Apple': 434,
 'Data Structures': 417,
 'MS': 373,
 'SQL': 300,
 'sci': 295,
 'STEP': 272,
 'GitHub': 271,
 'IBM': 268,
 'WLB': 266,
 'Meta': 260,
 'Capital One': 241,
 'Software Engineering': 237,
 'linear': 236,
 'OA2': 227,
 'MIT': 224,
 'YouTube': 223,
 'CSS': 221,
 'AP': 218,
 'HRT': 215,
 'Hackerrank': 215,
 'Data Science': 210,
 'FBU': 209,
 'Stanford': 199,
 'CodeSignal': 196,
 'Intro': 187,
 'Google STEP': 186,
 'OA': 184,
 'TA': 181,
 'Bachelor': 179,
 'HackerRank': 177,
 'CMU': 174,
 'AWS': 173,
 'Goldman Sachs': 163,
 'google': 160,
 'University': 159,
 'API': 158,
 'FAANG': 152,
 'CC': 151,
 'GRE': 151,
 'Walmart': 137,
 'Purdue': 136,
 'PS': 135,
 'CSE': 134,
 'AI/ML': 130,
 'Glassdoor': 130,
 'Tesla': 129,
 'microsoft': 127,
 'Big N': 125,
 'Harvard': 123,
 'TikTok': 119,
 'BA': 117,
 'UCSD': 116,
 'COVID': 115,
 'MSCS': 114,
 

In [2]:
from transformers import AutoTokenizer
import nltk
import spacy
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
import string
from nltk import pos_tag

# Load tokenizer for a specific pre-trained model
tokenizer = AutoTokenizer.from_pretrained('mwkby/distilbert-base-uncased-sentiment-reddit-crypto')
# Load tokenizer and spacy model for a specific pre-trained model
nlp = spacy.load("en_core_web_sm")

def tokenize(text):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)
    
    return tokens

def filter_nouns_and_verbs(words):
    # Use NLTK to tag each word with its part of speech
    tagged_words = pos_tag(words)
    
    # Filter out non-noun and non-verb words
    filtered_words = [word for word, tag in tagged_words if tag.startswith('N') or tag.startswith('V')]
    
    return filtered_words

def remove_stop_words(tokens):
    # Remove stopwords using NLTK
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Print the resulting list of tokens
    return filtered_tokens

def clean_tokens(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if all(c in string.ascii_letters for c in token)]
    filtered_tokens = remove_stop_words(tokens)
    filtered_tokens = filter_nouns_and_verbs(filtered_tokens)
    return filtered_tokens

def get_word_frequency(texts, top_k=10):
    words_list = []
    for text in texts:
        tokens = clean_tokens(text)
        words_list.append(tokens)
    # Flatten the list of lists into a single list
    words_flatten = [word for words in words_list for word in words]
    
    # Count the frequency of each word
    word_counts = Counter(words_flatten)
    
    # Sort the words by frequency in descending order
    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_words[:10]
# Define some example text
texts = ["The quick brown fox jumps over the lazy dog."]
get_word_frequency(texts)

[nltk_data] Downloading package stopwords to /Users/hieun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('brown', 1), ('fox', 1), ('jumps', 1), ('dog', 1)]

In [5]:
def filter_text_data(data):
    new_data = []
    for text in data:
        tokens = clean_tokens(text)
        string = " ".join(tokens)
        new_data.append(string)
    return new_data

In [51]:
from sentence_transformers import SentenceTransformer
import scipy.spatial

# Load a pre-trained sentence transformer model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Define two example sentences to compare
sentence1 = 'Managers at ggl is smart and engineering-oriented'
sentence2 = 'Managers at in tech is smart and engineering-oriented'

# Encode the sentences using the sentence transformer model
embeddings = model.encode([sentence1, sentence2])

# Calculate the cosine similarity between the sentence embeddings
similarity = 1 - scipy.spatial.distance.cosine(embeddings[0], embeddings[1])

# Print the similarity score
print('Similarity score:', similarity)


Similarity score: 0.8884086012840271


In [23]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = [post['selftext'] for post in data[-2000:]]
corpus_embeddings = embedder.encode(corpus)
print("Finish encode corpus")
# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
print("Finish fitting corpus embeddings")

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("CLUSTER ", i+1)
#     for i,text in enumerate(cluster):
#         print(f"{i}. {text}")
    print(get_word_frequency(cluster))
    print("----------------------------------")

# Clustering text, then takes top frequency words from each cluster


Finish encode corpus
Finish fitting corpus embeddings
CLUSTER  5
[('internship', 122), ('summer', 70), ('get', 63), ('companies', 58), ('year', 44), ('applying', 34), ('experience', 31), ('resume', 29), ('know', 25), ('school', 23)]
----------------------------------
CLUSTER  3
[('classes', 57), ('school', 56), ('semester', 53), ('internship', 51), ('year', 41), ('time', 37), ('taking', 36), ('cs', 35), ('course', 33), ('take', 33)]
----------------------------------
CLUSTER  18
[('deleted', 339)]
----------------------------------
CLUSTER  13
[('recruit', 26), ('got', 20), ('email', 20), ('interview', 19), ('get', 18), ('anyone', 16), ('know', 14), ('year', 14), ('weeks', 14), ('applied', 13)]
----------------------------------
CLUSTER  9
[('companies', 14), ('resume', 12), ('time', 11), ('career', 10), ('get', 9), ('company', 9), ('know', 8), ('year', 8), ('interview', 8), ('x', 8)]
----------------------------------
CLUSTER  10
[('interview', 169), ('questions', 41), ('google', 37),

In [52]:
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

# texts = ["Apple is looking at buying U.K. startup for $1 billion", 
#          "Google announces new AI tool for healthcare industry",
#          "Amazon Web Services outage causes major disruptions for businesses"]
texts = [post['selftext'] for post in data[:5000]]

orgs = {}
for text in tqdm(texts):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            orgs[ent.text] = 1 if ent.text not in orgs else orgs[ent.text] + 1

100%|███████████████████████████████████████| 5000/5000 [01:11<00:00, 70.07it/s]


In [92]:
# Print the topics and their top words
(topic_model.get_topic_freq())

,Topic,Count
0,0,77
1,1,23


In [113]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,240,0_to_the_and_in
1,1,34,1_removed_deleted__
2,2,26,2_all_that_is_


In [28]:
sorted_orgs = dict(sorted(orgs.items(), key=lambda item: item[1], reverse=True))

sorted_orgs

{'CS': 26,
 'AI': 3,
 'SQL': 3,
 'Amazon': 2,
 'CC': 2,
 'MacBook': 2,
 'GPA': 2,
 'Software Engineer': 1,
 'the University of Illinois Urbana-Champaign': 1,
 'the University of Michigan Ann Arbor': 1,
 'Data Science or Computer Engineering': 1,
 'CE': 1,
 'UIUC': 1,
 'UIUC CS': 1,
 'Cat Digital': 1,
 'GPT': 1,
 'Google': 1,
 'Ramp': 1,
 'SWE': 1,
 'Time': 1,
 'The Odin Project': 1,
 'Media Solutions': 1,
 'Auth': 1,
 'Greek Sexualities': 1,
 '2D': 1,
 'Searching and Sorting': 1,
 'ML': 1,
 'Stackoverflow': 1,
 'nyu': 1,
 'BlackRock': 1,
 'Physics': 1,
 'Physics/eng': 1,
 'Company A': 1,
 'Team A': 1,
 'SMU': 1,
 'Southern Methodist University': 1,
 'TX': 1,
 'PasteBin': 1,
 'Santa Clara Square Apartments': 1,
 'Whole Foods': 1,
 'Chase': 1,
 'Security Deposit': 1,
 'Amenity': 1,
 'Berkeley': 1,
 'UI/UX': 1,
 'Internships': 1,
 'ANN': 1,
 'Masters': 1,
 'RA': 1,
 'UMass Amherst MS CS': 1,
 'QA': 1,
 'UC Riverside': 1,
 'Hirevue': 1,
 'wouldn;t': 1,
 'Sac State': 1,
 'CSU': 1,
 'UC Merc

In [48]:
word_pair_distance("JP Morgan", "Google", model)

0.5882711

In [50]:
new_orgs = set()
for text in tqdm(orgs.keys()):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            new_orgs.add(ent.text)
len(new_orgs)

100%|██████████████████████████████████████| 3719/3719 [00:11<00:00, 334.30it/s]


2520

In [59]:
len(sorted_orgs)
# Retrieve all the post
# Label the post with the associated company
# Traverse CommentForest to extract all comments from each post
# Perform sentiment analysis


3719